Bu çalışmada Apache Beam ve Tensorflow kullanarak bir model oluşturulacak, eğitilecek (train), daha sonra eğitim sonuçları değerlendirilecek (evaluation) ve model ile tahminler(prediction) yapılacak.

Çalışmada karbon, hidrojen, oksijen v nitrojen atomlarının sayısına dayalı olarak moleküler enerji tahmini yapabildiğimiz bir veri seti kullanılacak.

Bütün işlemleri gerçekleştirirken takip edeceğimiz iş akışı (pipeline) dört aşamadan oluşmaktadır:

1- Veri Çıkarımı

2- Verilerin ön işlenmesi

3- Model eğitimi

4- Tahmin yapmak

In [1]:
# İndirilecek gerekli paketlerin Python 3.7 sürümüne ihtiyacı olduğundan versiyonu düşürüyoruz.
!apt-get install python3.7 python3.7-distutils python3-pip

# Colab ortamının Python 3.7 ile çalışabilmesi için konfigürasyon yapıyoruz.
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 100

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib mailcap mime-support python3-setuptools python3-wheel
  python3.7-lib2to3 python3.7-minimal
Suggested packages:
  python-setuptools-doc python3.7-venv binfmt-support
The following NEW packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib mailcap mime-support python3-pip python3-setuptools
  python3-wheel python3.7 python3.7-distutils python3.7-lib2to3 python3.7-minimal
0 upgraded, 11 newly installed, 0 to remove and 24 not upgraded.
Need to get 6,688 kB of archives.
After this operation, 28.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 mime-support all 3.66 [3,696 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 pyt

In [2]:
# veri dosyasını indiriyorum
!wget https://github.com/https-deeplearning-ai/machine-learning-engineering-for-production-public/raw/main/course4/week2-ungraded-labs/C4_W2_Lab_4_ETL_Beam/data/molecules.tar.gz


--2024-01-06 18:16:06--  https://github.com/https-deeplearning-ai/machine-learning-engineering-for-production-public/raw/main/course4/week2-ungraded-labs/C4_W2_Lab_4_ETL_Beam/data/molecules.tar.gz
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/https-deeplearning-ai/machine-learning-engineering-for-production-public/main/course4/week2-ungraded-labs/C4_W2_Lab_4_ETL_Beam/data/molecules.tar.gz [following]
--2024-01-06 18:16:06--  https://raw.githubusercontent.com/https-deeplearning-ai/machine-learning-engineering-for-production-public/main/course4/week2-ungraded-labs/C4_W2_Lab_4_ETL_Beam/data/molecules.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connec

In [3]:
# sıkıştırılmış tar dosyasını açıyorum
!tar -xvzf molecules.tar.gz --one-top-level

data-extractor.py
predict.py
preprocess.py
pubchem/
pubchem/__init__.py
pubchem/sdf.py
pubchem/pipeline.py
trainer/
trainer/task.py
trainer/__init__.py


Veri çıkarımı için data-extractor.py, önişleme için preprocess.py, eğitim için trainer/task.py ve prediction için predict.py dizinleri bulunmaktadır.

In [4]:
# paketleri indirelim
!pip install tensorflow-transform==1.12
!pip install dill==0.3.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.8/439.8 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/25.6 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 KB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 659.0/659.0 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 KB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 KB 1.6 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.1.1
    Uninstalling dill-0.3.1.1:
      Successfully uninstalled dill-0.3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.3 which is incompatible.


Bu notebook'un python sürümünü ve konfigürasyonda sürümlerini değiştirdiğimiz için runtime'ı yeniden başlatmak hata almamıza sebep olabilir. Yeniden başlatmadan önce bağlantıyı kesip runtime'ı silmemiz ve daha sonra baştan başlatmamız daha sağlıklı olacaktır.

In [5]:
WORK_DIR= "results"

### Veri Çıkarımı

Veri seti SDF dosya formatında depolanmakta ve National Center for Biotechnology Information (FTP Source) 'tan çekilmektedir.

data-extractor.py dosyası SDF dosyasını çıkarıp bahsedilen SDF sıkıştırılmış formatı açar.

In [6]:
# print the help documantation
!python ./molecules/data-extractor.py --help

2024-01-06 18:17:57.827624: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-06 18:17:57.997311: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-01-06 18:17:57.997371: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-01-06 18:17:59.065027: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7:

In [7]:
# run the data extractor
!python ./molecules/data-extractor.py --max-data-files 1 --work-dir={WORK_DIR}

2024-01-06 18:18:00.863213: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-06 18:18:01.031438: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-01-06 18:18:01.031480: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-01-06 18:18:02.104638: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7:

In [8]:
# çalışılan dizini listeleyelim
!ls {WORK_DIR}

data


In [9]:
# veri setinden bir veriyi yazdırmak için aşağıdaki komutu kullanmak gerekir.
!sed '/$$$$/q' {WORK_DIR}/data/00000001_00025000.sdf

1
  -OEChem-04092302373D

 31 30  0     1  0  0  0  0  0999 V2000
    0.3387    0.9262    0.4600 O   0  0  0  0  0  0  0  0  0  0  0  0
    3.4786   -1.7069   -0.3119 O   0  5  0  0  0  0  0  0  0  0  0  0
    1.8428   -1.4073    1.2523 O   0  0  0  0  0  0  0  0  0  0  0  0
    0.4166    2.5213   -1.2091 O   0  0  0  0  0  0  0  0  0  0  0  0
   -2.2359   -0.7251    0.0270 N   0  3  0  0  0  0  0  0  0  0  0  0
   -0.7783   -1.1579    0.0914 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.1368   -0.0961   -0.5161 C   0  0  2  0  0  0  0  0  0  0  0  0
   -3.1119   -1.7972    0.6590 C   0  0  0  0  0  0  0  0  0  0  0  0
   -2.4103    0.5837    0.7840 C   0  0  0  0  0  0  0  0  0  0  0  0
   -2.6433   -0.5289   -1.4260 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.4879   -0.6438   -0.9795 C   0  0  0  0  0  0  0  0  0  0  0  0
    2.3478   -1.3163    0.1002 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.4627    2.1935   -0.0312 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.6678    3.1549    

### Ön işleme

Verileri ön işlemek için Apache Beam iş akışını kullanan prepocess.py dosyasını kullanacağız.

İş akışı aşağıdaki adımlardan oluşmaktadır:

 1. Çıkarılan SDF dosyalarını okur ve ayrıştırır.
 2. Dosyalardaki moleküllerin her birindeki farklı atom sayılarını sayar.
 3. tf.Transform'u kullanarak değerleri 0-1 arasındaki değerlere normalleştirir.
 4. Veri setini bir eğitim seti ve bir değerlendirme seti olarak bölümlere ayırır.
 5. İki veri setini TFRecords nesneleri olarak yazar.


Apache Beam verileri tek başlarına etkili bir şekilde işleyebilir ancak veri setindeki bütün verileri baştan sona işlemek sadece Apache Beam kullanılarak kolayca halledilemez bu yüzden tf.Transform kullanılması iyidir. Bu nedenle kod, molekülleri okumak ve biçimlendirmek ve her moleküldeki atomları saymak için Apache Beam dönüşümlerini kullanır. Daha sonra verileri normalleştirmek için genel min ve genel max sayıları tf.Transform ile bulunur.